# Data Preprocessing
:label:`sec_pandas`

So far, we have been working with synthetic data
that arrived in ready-made tensors.
However, to apply deep learning in the wild
we must extract messy data
stored in arbitrary formats,
and preprocess it to suit our needs.
Fortunately, the *pandas* [library](https://pandas.pydata.org/)
can do much of the heavy lifting.
This section, while no substitute
for a proper *pandas* [tutorial](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html),
will give you a crash course
on some of the most common routines.

## Reading the Dataset

Comma-separated values (CSV) files are ubiquitous
for the storing of tabular (spreadsheet-like) data.
In them, each line corresponds to one record
and consists of several (comma-separated) fields, e.g.,
"Albert Einstein,March 14 1879,Ulm,Federal polytechnic school,field of gravitational physics".
To demonstrate how to load CSV files with `pandas`,
we (**create a CSV file below**) `../data/house_tiny.csv`.
This file represents a dataset of homes,
where each row corresponds to a distinct home
and the columns correspond to the number of rooms (`NumRooms`),
the roof type (`RoofType`), and the price (`Price`).


In [ ]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('''NumRooms,RoofType,Price
NA,NA,127500
2,NA,106000
4,Slate,178100
NA,NA,140000''')

Now let's import `pandas` and load the dataset with `read_csv`.


In [ ]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms RoofType   Price
0       NaN      NaN  127500
1       2.0      NaN  106000
2       4.0    Slate  178100
3       NaN      NaN  140000


## Data Preparation

In supervised learning, we train models
to predict a designated *target* value,
given some set of *input* values.
Our first step in processing the dataset
is to separate out columns corresponding
to input versus target values.
We can select columns either by name or
via integer-location based indexing (`iloc`).

You might have noticed that `pandas` replaced
all CSV entries with value `NA`
with a special `NaN` (*not a number*) value.
This can also happen whenever an entry is empty,
e.g., "3,,,270000".
These are called *missing values*
and they are the "bed bugs" of data science,
a persistent menace that you will confront
throughout your career.
Depending upon the context,
missing values might be handled
either via *imputation* or *deletion*.
Imputation replaces missing values
with estimates of their values
while deletion simply discards
either those rows or those columns
that contain missing values.

Here are some common imputation heuristics.
[**For categorical input fields,
we can treat `NaN` as a category.**]
Since the `RoofType` column takes values `Slate` and `NaN`,
`pandas` can convert this column
into two columns `RoofType_Slate` and `RoofType_nan`.
A row whose roof type is `Slate` will set values
of `RoofType_Slate` and `RoofType_nan` to 1 and 0, respectively.
The converse holds for a row with a missing `RoofType` value.


In [ ]:
inputs, targets = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       NaN           False          True
1       2.0           False          True
2       4.0            True         False
3       NaN           False          True


For missing numerical values,
one common heuristic is to
[**replace the `NaN` entries with
the mean value of the corresponding column**].


In [ ]:
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       3.0           False          True
1       2.0           False          True
2       4.0            True         False
3       3.0           False          True


## Conversion to the Tensor Format

Now that [**all the entries in `inputs` and `targets` are numerical,
we can load them into a tensor**] (recall :numref:`sec_ndarray`).


In [ ]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(targets.to_numpy(dtype=float))
X, y

(tensor([[3., 0., 1.],
         [2., 0., 1.],
         [4., 1., 0.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

## Discussion

You now know how to partition data columns,
impute missing variables,
and load `pandas` data into tensors.
In :numref:`sec_kaggle_house`, you will
pick up some more data processing skills.
While this crash course kept things simple,
data processing can get hairy.
For example, rather than arriving in a single CSV file,
our dataset might be spread across multiple files
extracted from a relational database.
For instance, in an e-commerce application,
customer addresses might live in one table
and purchase data in another.
Moreover, practitioners face myriad data types
beyond categorical and numeric, for example,
text strings, images,
audio data, and point clouds.
Oftentimes, advanced tools and efficient algorithms
are required in order to prevent data processing from becoming
the biggest bottleneck in the machine learning pipeline.
These problems will arise when we get to
computer vision and natural language processing.
Finally, we must pay attention to data quality.
Real-world datasets are often plagued
by outliers, faulty measurements from sensors, and recording errors,
which must be addressed before
feeding the data into any model.
Data visualization tools such as [seaborn](https://seaborn.pydata.org/),
[Bokeh](https://docs.bokeh.org/), or [matplotlib](https://matplotlib.org/)
can help you to manually inspect the data
and develop intuitions about
the type of problems you may need to address.


## Exercises

1. Try loading datasets, e.g., Abalone from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) and inspect their properties. What fraction of them has missing values? What fraction of the variables is numerical, categorical, or text?
1. Try indexing and selecting data columns by name rather than by column number. The pandas documentation on [indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) has further details on how to do this.
1. How large a dataset do you think you could load this way? What might be the limitations? Hint: consider the time to read the data, representation, processing, and memory footprint. Try this out on your laptop. What happens if you try it out on a server?
1. How would you deal with data that has a very large number of categories? What if the category labels are all unique? Should you include the latter?
1. What alternatives to pandas can you think of? How about [loading NumPy tensors from a file](https://numpy.org/doc/stable/reference/generated/numpy.load.html)? Check out [Pillow](https://python-pillow.org/), the Python Imaging Library.


## Discuss Handling Large Number of Categories

### Subtask:
Add a markdown cell to discuss strategies for dealing with data that has a very large number of categories or unique category labels.

Handling categorical data with a very large number of categories, often referred to as high cardinality, requires careful consideration.

*   **High Cardinality Categorical Features**: If a categorical feature has many distinct values (e.g., zip codes, product IDs), simply one-hot encoding can lead to a huge number of new features, increasing memory usage and potentially making models less efficient. Strategies include:
    *   **Target Encoding**: Replace each category with the mean of the target variable for that category.
    *   **Frequency Encoding**: Replace each category with its frequency in the dataset.
    *   **Grouping Categories**: Group less frequent categories into an "other" category.
    *   **Feature Hashing**: Convert categories into a fixed-size vector using a hash function.
    *   **Embedding**: For very high cardinality or text-like categories, embedding techniques (similar to word embeddings in NLP) can be used to represent categories in a lower-dimensional space.

*   **Unique Category Labels**: If each category label is unique for each data point (e.g., a unique user ID or transaction ID), these are typically not useful as direct features for a model designed to generalize. Including them would essentially be telling the model to memorize each instance. These columns are usually dropped or used only for indexing or joining with other data sources.

The decision of whether to include or process such features depends on the specific problem and dataset. Domain knowledge is crucial in determining if a high-cardinality feature carries predictive power or if unique identifiers should be used in a different way (e.g., for grouping or aggregating information from other tables).

## Discuss Alternatives to Pandas

### Subtask:
Add a markdown cell to discuss alternatives to pandas for data handling, including NumPy and Pillow.

While pandas is a powerful and widely used library for data manipulation and analysis, several alternatives exist, each with its strengths:

*   **NumPy**: As you've already seen, NumPy is fundamental for numerical operations in Python. While it doesn't offer the same level of data structure and manipulation capabilities as pandas DataFrames, it is highly efficient for numerical computations and array-based data. For loading numerical data directly from files, `numpy.load()` and `numpy.loadtxt()` are useful functions, particularly for binary or delimited text files.

*   **Dask**: Dask is a flexible library for parallel computing in Python. It integrates with libraries like NumPy and pandas, allowing you to work with larger-than-memory datasets and perform computations in parallel across multiple cores or machines. Dask DataFrames provide a distributed equivalent to pandas DataFrames.

*   **Apache Spark (with PySpark)**: Spark is a powerful unified analytics engine for large-scale data processing. PySpark is the Python API for Spark. It's designed for distributed computing and is well-suited for processing massive datasets across clusters of computers.

*   **Polars**: Polars is a DataFrame library written in Rust, which offers excellent performance for many data manipulation tasks. It's designed to be memory efficient and fast, particularly for operations that can be parallelized.

*   **Pillow (PIL)**: Pillow is the friendly fork of the Python Imaging Library (PIL). While not a general-purpose data manipulation library like pandas, it is the go-to library for working with image data in Python. It provides extensive capabilities for opening, manipulating, and saving various image file formats. When dealing with image datasets for machine learning, Pillow is often used to load and preprocess the images before they are converted into numerical tensors (e.g., using NumPy or PyTorch) for model input.

The choice of alternative depends on the specific data type, size, and the required operations. For numerical data that fits in memory and doesn't require complex indexing or heterogeneous columns, NumPy might suffice. For larger or more complex datasets, distributed computing frameworks like Dask or Spark, or performance-oriented libraries like Polars, might be more appropriate. For image data, Pillow is the standard.

[Discussions](https://discuss.d2l.ai/t/29)


# Task
Load the Abalone dataset from the UCI Machine Learning Repository, analyze its properties (missing values, variable types), demonstrate column selection by name, and discuss limitations of loading large datasets with pandas, handling large numbers of categories, and alternatives to pandas.

## Load and inspect a dataset

### Subtask:
Download a dataset from the UCI Machine Learning Repository (like Abalone), load it into a pandas DataFrame, and display basic information about it.


**Reasoning**:
The subtask requires downloading a dataset and loading it into a pandas DataFrame with specified column names. Then, display the head and info of the dataframe. These steps can be combined into a single code block.



In [2]:
import pandas as pd

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data'
column_names = ['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings']
abalone_df = pd.read_csv(url, names=column_names)

print("First 5 rows of the Abalone dataset:")
display(abalone_df.head())

print("\nConcise summary of the Abalone dataset:")
abalone_df.info()

First 5 rows of the Abalone dataset:


,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7



Concise summary of the Abalone dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4177 entries, 0 to 4176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4177 non-null   object 
 1   Length          4177 non-null   float64
 2   Diameter        4177 non-null   float64
 3   Height          4177 non-null   float64
 4   Whole_weight    4177 non-null   float64
 5   Shucked_weight  4177 non-null   float64
 6   Viscera_weight  4177 non-null   float64
 7   Shell_weight    4177 non-null   float64
 8   Rings           4177 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 293.8+ KB


## Analyze missing values

### Subtask:
Calculate and display the fraction of missing values in the dataset.


**Reasoning**:
Calculate the fraction of missing values for each column and display the result.



In [3]:
missing_values_per_column = abalone_df.isnull().sum()
total_entries = len(abalone_df)
fraction_missing = missing_values_per_column / total_entries
print("Fraction of missing values per column:")
print(fraction_missing)

Fraction of missing values per column:
Sex               0.0
Length            0.0
Diameter          0.0
Height            0.0
Whole_weight      0.0
Shucked_weight    0.0
Viscera_weight    0.0
Shell_weight      0.0
Rings             0.0
dtype: float64


## Analyze variable types

### Subtask:
Determine and display the fraction of numerical, categorical, and text variables in the dataset.


In [4]:
numerical_cols = abalone_df.select_dtypes(include=['int64', 'float64']).shape[1]
object_cols = abalone_df.select_dtypes(include=['object']).shape[1]
total_cols = abalone_df.shape[1]

fraction_numerical = numerical_cols / total_cols
fraction_categorical_text = object_cols / total_cols

print(f"Fraction of numerical variables: {fraction_numerical:.2f}")
print(f"Fraction of categorical/text variables: {fraction_categorical_text:.2f}")

Fraction of numerical variables: 0.89
Fraction of categorical/text variables: 0.11


## Indexing by name

### Subtask:
Demonstrate indexing and selecting columns by name using the loaded dataset.


In [5]:
# Select a single column by name
single_column = abalone_df['Rings']
print("Head of the 'Rings' column:")
display(single_column.head())

# Select multiple columns by name
multiple_columns = abalone_df[['Length', 'Diameter', 'Height']]
print("\nHead of the 'Length', 'Diameter', and 'Height' columns:")
display(multiple_columns.head())

Head of the 'Rings' column:


,Rings
0,15
1,7
2,9
3,10
4,7



Head of the 'Length', 'Diameter', and 'Height' columns:


,Length,Diameter,Height
0,0.455,0.365,0.095
1,0.350,0.265,0.090
2,0.530,0.420,0.135
3,0.440,0.365,0.125
4,0.330,0.255,0.080


## Discuss Dataset Size Limitations

### Subtask:
Add a markdown cell to discuss the limitations of loading large datasets using pandas, considering time, representation, processing, and memory.

Loading very large datasets into a pandas DataFrame can face several limitations:

*   **Time to Read Data**: Reading extremely large files (gigabytes or terabytes) can take a significant amount of time, especially from slower storage mediums.
*   **Representation and Memory Footprint**: pandas DataFrames are stored in memory. For very large datasets, the entire dataset might not fit into the available RAM, leading to memory errors. The way pandas stores data, especially objects and strings, can be memory-intensive.
*   **Processing Performance**: Operations on large DataFrames can be slow. While pandas is optimized for many operations, certain tasks on massive datasets can still be computationally expensive and time-consuming.
*   **Single-Machine Limitation**: pandas is primarily designed for single-machine processing. For datasets that exceed the capacity of a single machine, distributed computing frameworks are necessary.

These limitations highlight the need for alternative tools and techniques when working with big data, which will be discussed later.